In [3]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm
import pickle

In [4]:
def get_mfcc(file_path):
    y, sr = librosa.load(file_path)  # read .wav file
    hop_length = math.floor(sr*0.010)  # 10ms hop
    win_length = math.floor(sr*0.025)  # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1, 1))
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0)  # O^r
    # return T x 36 (transpose of X)
    return X.T  # hmmlearn use T x N matrix

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir, f))
            for f in files if f.endswith(".wav")]
    return mfcc


def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50,
                    random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans

def initByBakis(nComp, bakisLevel):
        ''' init start_prob and transmat_prob by Bakis model ''' 
        startprobPrior = np.zeros(nComp)
        startprobPrior[0 : bakisLevel - 1] = 1./ (bakisLevel - 1)
         
        transmatPrior = getTransmatPrior(nComp, bakisLevel)
         
        return startprobPrior, transmatPrior

def getTransmatPrior(nComp, bakisLevel):
        ''' get transmat prior '''
        transmatPrior = (1. / bakisLevel) * np.eye(nComp)
         
        for i in range(nComp - (bakisLevel - 1)):
            for j in range(bakisLevel - 1):
                transmatPrior[i, i + j + 1] = 1. /  bakisLevel
                 
        for i in range(nComp - bakisLevel + 1, nComp):
            for j in range(nComp - i -j):
                transmatPrior[i, i + j] = 1. / (nComp - i)
        return transmatPrior

In [5]:
class_names = ["cothe", "duoc", "khong", "nguoi", "trong", "test_cothe", "test_duoc", "test_khong", "test_nguoi", "test_trong"]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join("data/cutted", cname))

# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
print("vectors", all_vectors.shape)
# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors)
print("centers", kmeans.cluster_centers_.shape)

for cname in class_names:
    # convert all vectors to the cluster index
    # dataset['one'] = [O^1, ... O^R]
    # O^r = (c1, c2, ... ct, ... cT)
    # O^r size T x 1
    dataset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in dataset[cname]])


Load cothe dataset
Load duoc dataset
Load khong dataset
Load nguoi dataset
Load trong dataset
Load test_cothe dataset
Load test_duoc dataset
Load test_khong dataset
Load test_nguoi dataset
Load test_trong dataset
vectors (13684, 36)
centers (10, 36)
centers (10, 36)


In [12]:
models = {}
for cname in class_names:
    if cname[:4] != 'test':
        class_vectors = dataset[cname]
        if cname == 'cothe':
            nComp = 4 * 3
        else:
            nComp = 3 * 3
        startprobPrior,transmatPrior = initByBakis(nComp=nComp,bakisLevel=3)
        hmm = hmmlearn.hmm.MultinomialHMM(
            n_components=nComp, random_state=15, n_iter=1000, verbose=True,
            startprob_prior=startprobPrior,
            transmat_prior=transmatPrior,
            init_params='ste',
            params='ste'
        )

        X = np.concatenate(dataset[cname])
        lengths = list([len(x) for x in dataset[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X, lengths=lengths)
        models[cname] = hmm
print("Training done")
print("Testing")

training class cothe
(3535, 1) [44, 47, 51, 44, 47, 64, 41, 47, 41, 37, 37, 37, 37, 47, 37, 41, 37, 34, 41, 37, 47, 41, 37, 44, 47, 41, 51, 67, 37, 41, 41, 37, 37, 34, 47, 31, 67, 47, 37, 37, 44, 41, 37, 44, 34, 44, 44, 34, 47, 51, 34, 34, 34, 37, 41, 51, 27, 31, 37, 34, 47, 44, 37, 44, 34, 47, 37, 37, 44, 37, 41, 74, 44, 37, 41, 41, 41, 51, 44, 61, 37, 41, 41, 37] 84
         1       -8088.3895             +nan
         2       -7505.7971        +582.5924
         3       -7413.5217         +92.2754
         4       -7213.0626        +200.4592
         5       -6754.7256        +458.3370
         6       -6089.4117        +665.3138
         7       -5669.2160        +420.1958
         8       -5294.9880        +374.2280
         9       -4830.9172        +464.0708
        10       -4552.2772        +278.6400
        11       -4402.6200        +149.6572
        12       -4307.4358         +95.1842
        13       -4225.7193         +81.7165
        14       -4172.7573         +52.9621

In [15]:
accuracy = {}
for cname in class_names:
    if cname[:4] != 'test':
        continue
    total_data = len(dataset[cname])
    true_cnt = 0
    # true result là tên chính xác của bộ test
    true_result = class_names[class_names.index(cname) % 5]
    for O in dataset[cname]:
        score = {cname: model.score(O, [len(O)]) for cname, model in models.items() if cname[:4] != 'test'}
        result = max(score, key=lambda k: score[k])
        isTrue = true_result == result
        # print(cname, score, result, isTrue)
        if isTrue:
            true_cnt += 1
    accuracy[cname] = true_cnt/total_data
print("\n\nMultinomialHMM accuracy:", accuracy)



MultinomialHMM accuracy: {'test_cothe': 1.0, 'test_duoc': 0.9285714285714286, 'test_khong': 0.8, 'test_nguoi': 1.0, 'test_trong': 0.9333333333333333}


In [16]:
with open("multinomial_hmm.pkl", "wb") as file:
    pickle.dump(models, file)
pickle.dump(kmeans, open("kmeans.pkl", "wb"))
print("Saved!")

Saved!
